# Python Hooking, Patching and Injection


[PPT Link](https://github.com/wjo1212/ChinaPyCon2016/blob/master/PythonHooking/PythonHooking.pdf)



## Agenda

1. Object and Mutability
2. System Level Hook
    - Import Hook
    - System Hook
3. Language Level Hook
    - Context Manager
    - Magic Methods
    - Decorator
    - class Decorator
    - Metaclass
4. Interpreter Level Hook
    - AST
    - Bytecodes
    - Frame Object

## 1. Object and Mutability

> Everything in Python are objects.

### Example 1. how to intercept std I/O (print to buffer instead of console)

In [1]:
print "abc"
print "xyz"
print "123"

abc
xyz
123


In [2]:
import sys
from StringIO import StringIO
output = StringIO()

# patching
stdout = sys.stdout
sys.stdout = output

print "abc"
print "xyz"
print "123"

no resuslts, actually written to buffer.

In [3]:
output.seek(0)
output.readlines()

['abc\n', 'xyz\n', '123\n']

In [20]:
# recover
sys.stdout = stdout

print "abc"
print "xyz"
print "123"

abc
xyz
123


More

In [ ]:
sys.stderr
sys.stdin
sys.displayhook

### Example 2. How to audit all file opening (when, who and how open)?

In [16]:
import os

os.chdir("/home/ubuntu/python-hooking/data")

open("./t1.txt", "w").close()
open("./t2.txt", "w").close()
open("./t3.txt", "w").close()
open("./t4.txt", "w").close()

In [17]:
import time
open_file_history = []

real_open = __builtins__.open

def my_open(name, *args, **kwargs):
    open_file_history.append(time.strftime("%H:%M:%S") + '" open "' + name + '"') # *
    print("detected file opening: " + name)
    return real_open(name, *args, **kwargs)

__builtin__.open = my_open

In [18]:
open("./t1.txt", "w").close()
open("./t2.txt", "w").close()
open("./t3.txt", "w").close()
open("./t4.txt", "w").close()

detected file opening: ./t1.txt
detected file opening: ./t2.txt
detected file opening: ./t3.txt
detected file opening: ./t4.txt


In [19]:
open_file_history

['03:49:23" open "./t1.txt"',
 '03:49:23" open "./t2.txt"',
 '03:49:23" open "./t3.txt"',
 '03:49:23" open "./t4.txt"']

In [21]:
# recover
__builtins__.open = real_open

## 2. System Level Hook

### 2.1 Import Hook

In [22]:
def import_module(mod_name):
    if mod_name in sys.modules:
        return sys.modules[mod_name]
    
    for dir_name in sys.path:
        print dir_name
        file_name = op.join(dir_name, mod_name + ".py")
        
        m = _exec_file(mod_name, file_name)
        sys.modules[mod_name] = m
        return m
    
    raise ImportError("...")

In [ ]:
import sys
sys.modules.keys()[:10]

### 2.1.1 How to make a external module importable externally (w/o code change)

In [31]:
!ls import_hook_lib

hello_pycon.py


1) by `$PYTHONPATH`/sys.path

In [38]:
import sys
sys.path.append('./import_hook_lib')

import hello_pycon

hello pycon!


2) via site.py

In [1]:
import site
site.addsitedir('./import_hook_lib')

import hello_pycon

hello pycon!


More about **site-packages**:

1. `sitecustomize.py` in `$PYTHONPATH` will be loaded automatically when Python startup.
2. `*.pth` in `site-packages` dir will be loaded automatically when Python startup.

```bash
$ ls ./import_hook_lib
pycon.pth    sitecustomize.py
$ cat sitecustomize.py
import site
site.addsitedir('/home/ubuntu/python-hooking/import_hook_lib`)
$ cat pycon.pth
print 'pycon'
$ export PYTHONPATH=`/home/ubuntu/python-hooking/import_hook_lib`:$PYTHONPATH
$ python -c "pass"
pycon
```

### 2.1.2 How many ways to import a library?

**method 1. import keyword**

In [2]:
import load_me

load me: I'm loaded!


**method 2. use `__import__` function**

In [1]:
load_me = __import__('load_me')

load me: I'm loaded!


**method 3. use imp**

could by pass sys.modules, import
(deprecated in Py3 by `importlib`, `importlib.reload`)

In [4]:
import imp

name = 'load_me'
fp, pathname, description = imp.find_module(name)
try:
    json = imp.load_module(name, fp, pathname, description)
finally:
    if fp:
        fp.close()
    
# note: the module is loaded again

load me: I'm loaded!


**method 4. use importlib**

replace imp in Py3 (recommended in Py3 than `__import__`)

In [1]:
import importlib
json = importlib.import_module('load_me')

load me: I'm loaded!


### 2.1.3 How to audit module importing?

**How to monitor those importing?**


```python
import md5
import json
import socket


> Hook `__import__`

Won't work in IPython, IPython already hooked `__import__`

```python
real_importer = __import__

def my_importer(*args, **kwargs):
    print("** importing: " + args[0])
    return real_importer(*args, **kwargs)



__builtins__.__import__ = my_importer

import os
import importlib
import imp

importlib.import_module('json')


def imp_load(name):
    fp, pathname, description = imp.find_module(name)
    try:
        json = imp.load_module(name, fp, pathname, description)
    finally:
        if fp:
            fp.close()

imp_load('load_me')

```

import, importlib are hooked, imp is NOT hooked.

### 2.1.4 How to replace installed module with local version?

add `sys.path`

In [1]:
! cat import_hook_lib/urllib2.py

def urlopen(*args, **kwargs):
    print("dummy urlopen: {}".format(str(args)))
    print("** do something cool **")
    return args[0]


In [3]:
import sys
sys.path.insert(0, './import_hook_lib')

# urllib2 module is hooked.

import urllib2
urllib2.urlopen('http://localhost:8888/notebooks')

dummy urlopen: ('http://localhost:8888/notebooks',)
** do something cool **


'http://localhost:8888/notebooks'

### 2.1.5 How to import a non-existing module by providing temporary ones

In [4]:
class PyCon(object):
    def __str__(self):
        return "hello PyCon China 2016!"
    
import pycon2016 # if not exist, provide a object of PyCon()

ImportError: No module named pycon2016

hook `sys.meta_path`

Yet, another hooking method as `__import__`

In [2]:
import sys

class Watcher(object):
    @classmethod
    def find_module(cls, name, path, target=None):
        print("Imorting: {}".format(name))
        return None # Note: bypass to other Finder/Loader

# insert into sys.meta_path
sys.meta_path.insert(0, Watcher)

# test
import mailbox

Imorting: mailbox
Imorting: mailbox
Imorting: email.generator
Imorting: email.generator
Imorting: email.header
Imorting: email.header
Imorting: rfc822
Imorting: rfc822


In [1]:
import sys

# note the PyCon class
class PyCon(object):
    def __str__(self):
        return "hello PyCon China 2016!"

class MyModuleLoader(object):
    @classmethod
    def find_module(cls, name, path, target=None):
        if name == 'pycon2016':
            return cls # Note
        return None

    @classmethod
    def load_module(cls, name):
        if name == 'pycon2016':
            return PyCon()
        raise ImportError("pycon")

# insert into sys.meta_path
sys.meta_path.insert(0, MyModuleLoader)

# test
import pycon2016
print pycon2016

hello PyCon China 2016!


### 2.1.6 How to support a remote virtual repo in sys.path?

```python
import sys
sys.path.append("box://mycompany.repo.com/repo/team")

import some_module # automatically import the some_module from box repo
```

**hook `sys.path_hook`**

for specific file path, folder or repo (e.g. zip)

In [2]:
import sys
sys.path_hooks

[zipimport.zipimporter]

In [3]:
[p for p in sys.path if 'zip' in p] # note the zip file

[]

**Examples: one path hook to support lib hosted on remote repo (e.g. AWS s3)**

In [4]:
import sys

class Dummy(object):
    def __str__(self):
        return "Hello PyCon 2016"

class S3ImportParser(object):
    KEY = 's3://'
    def __init__(self, path_entry):
        if path_entry.startswith(self.KEY):
            print "Handle: " + path_entry
            self.path_repo = path_entry
            return

        # raise ImportError means passing to other Loader
        raise ImportError()

    def find_module(self, fullname, path=None):
        print 'Search for "%s" on %s' % (fullname, self.path_repo)
        return self
        
    def load_module(self, fullname):
        print 'Load for "%s"' % fullname
        return Dummy()
    
    
# test
sys.path_hooks.append(S3ImportParser)

# suppose there's a S3 repo in sys.path
sys.path.append("s3://mycompany.repo.com/repo/team")

import some_module

print "-" * 30
print str(some_module)

### 2.2 System Hook

### 2.2.1 How to hook system exit event and do some clean-up

**exit hook - atexit**

cat ./sys_hook/exit_hook.py

```python
from threading import Thread, current_thread

import atexit

def exit0(*args, **kwarg):
    print '** exit0', current_thread().getName(), args, kwarg

def exit1():
    print '** exit1', current_thread().getName()
    raise Exception, 'exit1'

def exit2():
    print '** exit2' , current_thread().getName()


atexit.register(exit0, 1, 2, a=1)
atexit.register(exit1)
atexit.register(exit2)

@atexit.register
def exit3():
    print '** exit3', current_thread().getName()

if __name__ == '__main__':
    print '** main', current_thread().getName()
```

In [7]:
!python ./sys_hook/exit_hook.py

** main MainThread
** exit3 Dummy-1
** exit2 Dummy-1
** exit1 Dummy-1
Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/usr/lib/python2.7/atexit.py", line 24, in _run_exitfuncs
    func(*targs, **kargs)
  File "./sys_hook/exit_hook.py", line 11, in exit1
    raise Exception, 'exit1'
Exception: exit1
** exit0 Dummy-1 (1, 2) {'a': 1}
Error in sys.exitfunc:
Traceback (most recent call last):
  File "/usr/lib/python2.7/atexit.py", line 24, in _run_exitfuncs
    func(*targs, **kargs)
  File "./sys_hook/exit_hook.py", line 11, in exit1
    raise Exception, 'exit1'
Exception: exit1


**Note**:

1. Sequence: LIFO
2. Threading Context: dummy-1
3. Exception: overcome
4. parameters

### 2.2.2 How to know if some specific exceptions happens

`sys.settrace`

**capture all system events**

Examples: capture all exception events

In [1]:
import sys, traceback
stop = False
def trace(frame, event, args):
    if stop:
        return

    if event == "exception":
        print "capture exception: ", args[0], args[1]
        print "----" * 10

    return trace
    
sys.settrace(trace)

import md_exception

stop = True

capture exception:  <type 'exceptions.KeyError'> ('abc',)
----------------------------------------
***complete import:  md_exception.py


## 3. Language Level Hook (syntax sugar)

### 3.1 How to control a block's enter and exit

> logging, timer, resource control etc.

**Context Manager**

Provide hook a "Enter" especially "Exit" event for a block of code

In [10]:
import time

class MyTimer(object):
    def __init__(self, tag='default'):
        self.tag = tag
        
    def __enter__(self,):
        self.start = time.time()
        
    def __exit__(self, exc, val, trace):
        self.end = time.time()
        print '*** performance for "' + self.tag + '" ***'
        print " {} seconds".format(self.end - self.start)
        return True

In [11]:
def test1(x, max):
    try:
        1/(max-x-1)
    except ZeroDivisionError as e:
        raise
        
def test2(x, max):
    1/(max-x-1)
    
with MyTimer("function call"):
    for x in range(10000000):
        test1(x, 10000000)
        
print "-" * 30
    
with MyTimer("function call with try block"):
    try:
        for x in range(10000000):
            test2(x, 10000000)
    except ZeroDivisionError as ex:
        raise

*** performance for "function call" ***
 9.50240397453 seconds
------------------------------
*** performance for "function call with try block" ***
 9.74168086052 seconds


**Note:**

1. `__exit__` is called even exception happens (return True to overcome)
2. quite suitable for resource management and scope management

### 3.2 Magic Methods

> Provide hook all kinds of operations around an object

![Magic Method](media/MagicMethod.png)

### 3.2.1 How to support "Lazy property" via chain calling

```python
# Note: consider a 10GB table
students = AzureTable('user1.storage.azure.com/table/students')

# Only get the data when be accessed
print students.xiaoMing.Address.data
print students.Jim.birthday.data
```

In [2]:
class AzureTable(object):
    def __init__(self, url, row=None, col=None, level='table'):
        self.__url = url
        self.__row = row
        self.__col = col
        self.__level = level

    def _fetch(self):
        print '*** Downloading from\n\t"{}/{}/{}"'.format(self.__url,
                                                          self.__row,
                                                          self.__col)
        print '*** Downloading Complete'
        return "Hello PyCon 2016: {}.{}".format(self.__row, self.__col)

    def __getattr__(self, item):
        if self.__level == 'table':
            return AzureTable(self.__url, row=item, level='row')
        elif self.__level == 'row':
            return AzureTable(self.__url, row=self.__row, col=item, level='col')

        if item == 'data':
            return self._fetch()

students = AzureTable('user1.storage.azure.com/table/students')
print students.xiaoMing.Address.data
print "-" * 30
print students.Jim.birthday.data

*** Downloading from
	"user1.storage.azure.com/table/students/xiaoMing/Address"
*** Downloading Complete
Hello PyCon 2016: xiaoMing.Address
------------------------------
*** Downloading from
	"user1.storage.azure.com/table/students/Jim/birthday"
*** Downloading Complete
Hello PyCon 2016: Jim.birthday


### 3.2.2 How to auditting property access for an 3rd party object and how to provide default value for 3rd party object with limited `__slot__`?

In [4]:
from collections import namedtuple

Point = namedtuple('Point', ['x', 'y'])

p = Point(x=1, y=2)

print p.x # How to monitor the access?
print p.y

# print p.z # How to provide default value?
# p.z = 10 # Note: cannot do this!

1
2


In [5]:
from collections import namedtuple
Point = namedtuple('Point', ['x', 'y'])

def monitor(self, item):
    print "** accessing :" + item
    return super(Point, self).__getattribute__(item)

Point.__getattribute__ = monitor

p = Point(x=1, y=2)
print p.x
print p.y

def handle_non_exist(self, item):
    print "** handling :" + item
    return str(item)

Point.__getattr__ = handle_non_exist

print p.z

** accessing :x
1
** accessing :y
2
** accessing :z
** handling :z
z


### 3.2.3 How to validate property value setting for an object transparently?

**Descriptors**

```python
p = People(age=1, height=200) # allow
p.age = 10 # allow!
p = People(age=-1, height=200) # disallow!
p.age = 0 # disallow!
```

Descriptor is a more low-granularity acess control

In [10]:
class IntegerProperty(object):
    def __init__(self, mi=None, mx=None):
        self.min = mi
        self.max = mx

    def __get__(self, obj, objtype):
        return self.val

    def __set__(self, obj, val):
        if (self.min is None or self.min <= val) \
            and (self.max is None or val <= self.max):
            self.val = val
        else:
            raise ValueError("value is out of range")
    # def __delete__(...)


class People(object):
    age = IntegerProperty(1, 130)
    height = IntegerProperty(1, 230)

    def __init__(self, age, height):
        self.age = age
        self.height = height

p = People(age=1, height=200)
p.age = 10

# Demo: do some typing here

p.age = 0
# p = People(age=-1, height=200)
# del p.age

ValueError: value is out of range

### 3.2.4 Operator overwritting

### Example 1. How to support Scala Lambda in Python

```python
map(_ * 2, xrange(4))
# get: [0, 2, 4, 6]

map(10 + _, xrange(4))
# get: [10, 11, 12, 13]
```

In [11]:
class Call(object):
    def __mul__(self, other):
        return lambda a: a * other

    def __radd__(self, other):
        return lambda a: a + other

_ = Call()

print map(_ * 2, xrange(4))
print map(10 + _, xrange(4))

[0, 2, 4, 6]
[10, 11, 12, 13]


### Example 2. How to support stream stype workflow operation?

```python
t1, t2, t3, t4, t5, t6, t7 = (Task('t' + str(x)) for x in range(1,8))

t1 >> t2 >> t3 >> t4
t2 >> t5 >> t6
t1 >> t7

Dag.draw_dag(t1)

# get:
"""
    + t1
        + t2
            + t3
                + t4
            + t5
                + t6
        + t7
"""
```

In [13]:
class Task(object):
    def __init__(self, name):
        self.name = name
        self.post_tasks = []

    def __rshift__(self, task):
        self.post_tasks.append(task)
        return task

    def __str__(self):
        return self.name


t1, t2, t3, t4, t5, t6, t7 = (Task('t' + str(x)) for x in range(1,8))
t1 >> t2 >> t3 >> t4
t2 >> t5 >> t6
t1 >> t7

class Dag(object):
    @staticmethod
    def draw_task(task, level):
        print "    " * (level + 1) + " + " + str(task)

    @staticmethod
    def draw_dag(task, level=0):
        Dag.draw_task(task, level)
        for n in task.post_tasks:
            Dag.draw_dag(n, level+1)
            
Dag.draw_dag(t1)

     + t1
         + t2
             + t3
                 + t4
             + t5
                 + t6
         + t7


### 3.3.1 How to monitor function calling and do something like logging, timing?

In [15]:
import wrapt, time

def logger(prefix='*'):
    @wrapt.decorator
    def _logger(fn, instance, args, kwargs):
        print '{} Enter "{}"'.format(prefix, fn.func_name)
        ret = fn(*args, **kwargs)
        print '{} Exit "{}"'.format(prefix, fn.func_name)
        return ret

    return _logger

@wrapt.decorator
def timer(fn, instance, args, kwargs):
    t1 = time.time()
    ret = fn(*args, **kwargs)
    print "* Time consumed: {} seconds".format(time.time() - t1)
    return ret


@timer
@logger("+")
def do_job():
    time.sleep(0.5)
    print "Hello PyCon 2016 China!"
    
do_job()

+ Enter "do_job"
Hello PyCon 2016 China!
+ Exit "do_job"
* Time consumed: 0.501806020737 seconds


### 3.3.2 How to make the function more robust by overcome some exceptions?

In [16]:
import wrapt

@wrapt.decorator
def ignore_error(fn, instance, args, kwargs):
    try:
        return fn(*args, **kwargs)
    except Exception as e:
        print "# Skip errors: {}".format(e)

@ignore_error
def do_job():
    print "Hello PyCon 2016 China!"
    raise ValueError("invalid")
    
do_job()

Hello PyCon 2016 China!
# Skip errors: invalid


### 3.3.3 How to add features to some functions like check auth for page rendering?

In [18]:
import wrapt
@wrapt.decorator
def check_auth(fn, instance, args, kwargs):
    if getattr(instance, 'need_login', False) \
        and not getattr(instance, 'session_key', ''):
        print "**** Permission Denied: {}".format(type(instance))
        return # Do nothing

    return fn(*args, **kwargs)

class HomePage(object):
    need_login = False

    @check_auth
    def show(self):
        print "** This is Home Page."

class AdminPage(object):
    need_login = True # check session key
    session_key = ''

    @check_auth
    def show(self):
        print "** This is Admin Page."

p1 = HomePage()
p2 = AdminPage()

p1.show()
p2.show()

** This is Home Page.
**** Permission Denied: <class '__main__.AdminPage'>


### 3.3.4 How to change function behavior: e.g. directly make it dummy?

In [19]:
import wrapt

@wrapt.decorator
def dummy(fn, instance, args, kwargs):
    return "Do nothing"

@dummy
def do_job():
    time.sleep(0.5)
    print "Hello PyCon 2016 China!"

do_job()

'Do nothing'

## 3.4 Class Decorator: Provide hook for  class construction/method calling

### 3.4.1 How to simply make a class's methods thread-safe

In [21]:
import time

class Task(object):
    def __init__(self):
        self.data = "xxx"

    def run1(self):
        data = self.data
        time.sleep(0.5)
        self.data = data + "111"

    def run2(self):
        data = self.data
        time.sleep(0.5)
        self.data = data + "222"

    def run3(self):
        data = self.data
        time.sleep(0.5)
        self.data = data + "333"



from threading import Thread

t = Task()

ts = [Thread(target=lambda j: j.run1(), args=(t, )),
        Thread(target=lambda j: j.run2(), args=(t, )),
        Thread(target=lambda j: j.run3(), args=(t, ))]

[s.start() for s in ts]
[s.join() for s in ts]

print "final data:", t.data
assert len(t.data) == 12, "t.data length should be 12 but is " + str(len(t.data))

final data: xxx333


AssertionError: t.data length should be 12 but is 6

In [22]:
import wrapt, time, threading, inspect
from threading import Thread

def synchronized(cls):
    lock = threading.RLock()

    @wrapt.decorator
    def _wrapper(fn, instance, args, kwargs):
        with lock:
            return fn(*args, **kwargs)

    for k, v in cls.__dict__.iteritems():
        if not k.startswith("__") and inspect.isfunction(v):
            setattr(cls, k, _wrapper(v))

    return cls


@synchronized
class Task(object):
    def __init__(self):
        self.data = "xxx"

    def run1(self):
        data = self.data
        time.sleep(0.5)
        self.data = data + "111"

    def run2(self):
        data = self.data
        time.sleep(0.5)
        self.data = data + "222"

    def run3(self):
        data = self.data
        time.sleep(0.5)
        self.data = data + "333"


t = Task()
ts = [Thread(target=lambda j: j.run1(), args=(t, )),
        Thread(target=lambda j: j.run2(), args=(t, )),
        Thread(target=lambda j: j.run3(), args=(t, ))]

[s.start() for s in ts]
[s.join() for s in ts]

print "final data:", t.data
assert len(t.data) == 12, "t.data length should be 12 but is " + str(len(t.data))

final data: xxx111222333


### about Decorator Overhead
use optimized wrapt for relative low performance impact

## 3.6 Metaclass

> Provide hook for class construction (somehow metaprogramming)

### 3.6.1 How to monitor class generation (in class hierarchy)

when, how, who and easily get all sub-class for a base class

In [25]:
class RegisterLeafClasses(type):
    def __init__(cls, name, bases, nmspc):
        super(RegisterLeafClasses, cls).__init__(name, bases, nmspc)
        if not hasattr(cls, 'registry'):
            cls.registry = set()
        cls.registry.add(cls)
        cls.registry -= set(bases) # Remove base classes

    def __str__(cls):
        if cls in cls.registry:
            return cls.__name__
        return cls.__name__ + ": " + ", ".join([sc.__name__ for sc in cls.registry])

class Color(object):
    __metaclass__ = RegisterLeafClasses

class Blue(Color): pass
class Red(Color): pass
class Green(Color): pass
class Yellow(Color): pass
print(Color)
class PhthaloBlue(Blue): pass
class CeruleanBlue(Blue): pass
print(Color)

Color: Red, Blue, Yellow, Green
Color: PhthaloBlue, Yellow, Red, Green, CeruleanBlue


### 3.6.2 How to make a class as final (cannot be inherited)

In [27]:
class final(type):
    def __init__(cls, name, bases, namespace):
        super(final, cls).__init__(name, bases, namespace)
        for klass in bases:
            if isinstance(klass, final):
                print "**debug** ", name, bases, namespace
                raise TypeError(str(klass.__name__) + " is final")

class A(object):
    pass

class B(A):
    __metaclass__= final

# compile error cause B is final
class C(B):
    pass

**debug**  C (<class '__main__.B'>,) {'__module__': '__main__'}


TypeError: B is final

### 3.6.3 Another way to decorate class

Example: Syncrhonized metaclass

In [28]:
import threading, inspect, collections
import wrapt

class SynchronizedClass(type):
    @classmethod
    def __prepare__(name, bases, **kwds):
        return collections.OrderedDict()

    def __new__(metacls, name, bases, namespace, **kwds):
        ret = type.__new__(metacls, name, bases, dict(namespace))

        ret.lock = threading.RLock()

        @wrapt.decorator
        def _wrapper(fn, instance, args, kwargs):
            with ret.lock:
                return fn(*args, **kwargs)

        for k, v in namespace.iteritems():
            if not k.startswith("__") and inspect.isfunction(v):
                setattr(ret, k, _wrapper(v))

        return ret

class Task(object):
    __metaclass__ = SynchronizedClass

    def __init__(self):
        self.data = "xxx"

    def run1(self):
        data = self.data
        time.sleep(0.5)
        self.data = data + "111"

    def run2(self):
        data = self.data
        time.sleep(0.5)
        self.data = data + "222"

    def run3(self):
        data = self.data
        time.sleep(0.5)
        self.data = data + "333"


t = Task()
ts = [Thread(target=lambda j: j.run1(), args=(t, )),
        Thread(target=lambda j: j.run2(), args=(t, )),
        Thread(target=lambda j: j.run3(), args=(t, ))]

[s.start() for s in ts]
[s.join() for s in ts]

print "final data:", t.data
assert len(t.data) == 12, "t.data length should be 12 but is " + str(len(t.data))

final data: xxx111222333


## 4. Interpreter Level Hook
### 4.1 AST


> Provide hook method to change the Syntax tree

![media](media/PythonAST.png)


In [1]:
from macropy.tracing import macros, require

try:
    require[3**2 + 4**2 != 5**2]
except AssertionError as e:
    print e
    
try:
    a = 10
    b = 2
    with require:
        a > 5
        a * b == 20
        a < 2
except AssertionError as e:
    print e

TypeError: Macro `require` illegally invoked at runtime; did you import it properly using `from ... import macros, require`?

## 4.2 Bytecodes

In [2]:
def hello():
    print "hello world"
    
hello()

hello world


Is it possible to hack the string "hello world"?

In [3]:
import dis
from byteplay import Code, LOAD_CONST

c = Code.from_code(hello.__code__)
print c.code


  2           1 LOAD_CONST           'hello world'
              2 PRINT_ITEM           
              3 PRINT_NEWLINE        
              4 LOAD_CONST           None
              5 RETURN_VALUE         



In [4]:
print c.code[0]
print c.code[1]

(SetLineno, 2)
(LOAD_CONST, 'hello world')


In [5]:
c.code[1] = (LOAD_CONST, "hello pycon 2016!")
hello.__code__ = c.to_code()

hello()

hello pycon 2016!


## 4.3 Frame Object

[Frame Hacks](http://feihonghsu.com/secrets/framehack/index.html)

### Example 1. how to support interpolate string?

In [6]:
name = 'Guido van Rossum'
places = 'Amsterdam', 'LA', 'New York', 'DC', 'Chicago',
s = """My name is ${'Mr. ' + name + ', Esquire'}.
I have visited the following cities: ${', '.join(places)}.
"""

print s

My name is ${'Mr. ' + name + ', Esquire'}.
I have visited the following cities: ${', '.join(places)}.



**Expected output**

```
My name is Mr. Guido van Rossum, Esquire.
I have visited the following ci ties: Amsterdam, LA, New York, DC, Chicago.
```

In [12]:
import sys, re

def getchunks(s):
    matches = list(re.finditer(r"\$\{(.*?)\}", s))
    
    if matches:
        pos = 0
        for match in matches:
            yield s[pos : match.start()]
            yield [match.group(1)]
            pos = match.end()
    yield s[pos:]

def interpolate(templateStr):
    framedict = sys._getframe(1).f_locals
    
    result = ''
    for chunk in getchunks(templateStr):
        if isinstance(chunk, list):
            result += str(eval(chunk[0], globals(), framedict))
        else:
            result += chunk
            
    return result

In [13]:
def foo():
    name = 'Guido van Rossum'

    places = 'Amsterdam', 'LA', 'New York', 'DC', 'Chicago',
    s = """My name is ${'Mr. ' + name + ', Esquire'}.
    I have visited the following cities: ${', '.join(places)}.
    """
    print interpolate(s)

foo()

My name is Mr. Guido van Rossum, Esquire.
    I have visited the following cities: Amsterdam, LA, New York, DC, Chicago.
    


### Example 2. Generating properties on a wrapper class

In [14]:
class Employee(object):
    def __init__(self):
        self._given = ""
        self._family = ""
        self.birth = ""

    given = property(
        lambda self: self._given,
        lambda self, v: setattr(self, '_given', v)
    )
    family = property(
        lambda self: self._family,
        lambda self, v: setattr(self, '_family', v)
    )
    birth = property(
        lambda self: self._birth,
        lambda self, v: setattr(self, '_birth', v)
    )


Use a frame hack to simplify the definition of properties:

```python
class Employee(object):
    properties(
        given  = 'GivenName',
        family = 'FamilyName',
        birth =  'DateOfBirth',
    )
```

In [18]:
import sys

def properties(**kwargs):
    frame = sys._getframe(1)
    for name, value in kwargs.items():
        frame.f_locals[name] = property(
            lambda self: value,
            lambda self, v: setattr(self, name, v)
            )


class Employee(object):
    properties(
        given  = 'GivenName',
        family = 'FamilyName',
        birth =  'DateOfBirth',
    )

e = Employee()
print(e.family)

FamilyName
